# Colorize

> Colorize text via ansi escape code

- Fore: foreground
- Back: background
- style: font style

In [ ]:
#| default_exp colorize

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.utils import *

In [ ]:
#| export

import contextlib
import io
import re
import sys

from colortextpy.ansicolor import Fore, Back, Style


class SystemStream:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            for file in (sys.stdout, sys.stderr):
                if '__' in repr(file):
                    raise RuntimeError(f'{file} has been modified')
            cls._instance = super().__new__(cls)
            cls.orig_stdout, cls.orig_stderr = sys.stdout, sys.stderr
        return cls._instance
    
    def __init__(self):
        self._affected = False

    def restore(self):
        sys.stdout, sys.stderr = self.orig_stdout, self.orig_stderr
        
    @property    
    def stdout(self): return self.orig_stdout
    
    @property
    def stderr(self): return self.orig_stderr
            
system_stream = SystemStream()

class ColorizedStream(contextlib.ContextDecorator):
    _regex_tag = re.compile('stdout|stderr')
    
    def __init__(self, fore=None, back=None, styles=None, autoreset=True, streams=('stdout', )):
        '''
        fore: str
                See Fore.available
        back: str
                See Back.available
        styles: str, tuple
                See Style.available
        autoreset: bool
        
        streams: str, tuple
            One of {stdout, stderr}, Could be tuple
        '''
        self.fore = self._set_color(fore, Fore)
        self.back = self._set_color(back, Back)
        self.style = (
            ''.join(self._set_color(s, Style) for s in styles) 
            if isinstance(styles, (tuple, list)) else 
            self._set_color(styles, Style)
        )
        self._global_flag = False
        self.autoreset = autoreset
        self._ori_reset = autoreset
        
        
        self.streams = streams if isinstance(streams, (list, tuple)) else (streams, )
        
        self.files = [getattr(sys, stream) for stream in streams]
        self.ori_files = []
        
             
    def _set_color(self, color, Color):
        if color is None: return ''
        elif '[' in color: return color
        elif color in Color: return Color[color]
        elif isinstance(color, tuple):
            if Color.name == 'fore':
                return colorstr.get_ansi('fg {0}, {1}, {2}'.format(*color))
            elif Color.name == 'back':
                return colorstr.get_ansi('bg {0}, {1}, {2}'.format(*color))
        else: return colorstr.get_ansi(color)
            
    def affect_global_stream(self):
        if not self._global_flag:
            self.__enter__()
            self._ori_reset = self.autoreset
            
        self.autoreset = False
        self._global_flag = True
        
    def unAffect_global_stream(self):
        self.__exit__()
        self._global_flag = False
        self.autoreset = self._ori_reset
        
    def __enter__(self):
        if not self.ori_files:
            for stream in self.streams:
                self.ori_files.append(getattr(system_stream, stream))
                setattr(sys, stream, self)

    def __exit__(self, *args):
        if self.ori_files:
            for stream, ori_file in zip(self.streams, self.ori_files):
                setattr(sys, stream, ori_file)
            self.ori_files = []

    def write(self, text):
        for file in self.ori_files:
            reset = Style.reset_all if self.autoreset else ''
            file.write(f'{self.style}{self.fore}{self.back}{text}{reset}')
            file.flush()
            
    def flush(self):
        for file in self.ori_files:
            file.flush()           
            
class AnsiColorizer:
    _regex_tag = re.compile(r"<([/\w\s,#]*)>")
    
    def __call__(self, text, fore=None, back=None, styles=None, raw=False, strip=False):
        '''
        text: str
        Some usage:
            text = 'something'
            1. blue text tag: f'<blue>{text}</fg>'
            2. blue text tag: f'<blue>{text}</blue>
            3. red text tag: f'<fg red>{text}</fg>'
            4. rgb text tag: f'<224, 224, 224>{text}</fg>'
            5. rgb text tag: f'<fg 224, 224, 224>{text}</fg>'
            6. blue background tag: f'<bg blue>{text}</bg>'
            7. HEX code text tag: f'<fg #FFFFFF>{text}</fg>'
            8. HEX code background tag: f'<bg #FFFFFF>{text}</bg>'
            9. style text tag: f'<bold>{text}</bold>'
            10. mix tags: f'<bold><yellow>{text}<bg #FA45BBB>test567</fg></bg></bold>'
        
        
        fore: str
                Foreground. See Fore.available
                Both Fore.red and 'red' are acceptable.
        back: str
                Background. See Back.available
                Both Back.red and 'red' are acceptable.
        styles: str, tuple
                See Style.available
                Both Style.bold and 'bold' are acceptable.
        '''        
        if any((fore, back, styles)):
            return (
                f'{self._set_color(fore, Fore)}'
                f'{self._set_color(back, Back)}'
                f'{"".join(self._set_color(s, Style) for s in styles) if isinstance(styles, (list, tuple)) else self._set_color(styles, Style)}'
                f'{text}'
                f'{Style.reset_all}'
            )
        if raw:     return text
        elif strip: return self.strip(text)
        else:       return self.colorize(text)
    
    def _set_color(self, color, Color):
        if color is None:
            return ''
        else:
            return color if '\033[' in color else Color[color]
        
    def get_ansi(self, tag):
        tag = tag.lower()

        # Substitute on a direct match.
        if tag in Style:
            return Style[tag]
        elif tag in Fore:
            return Fore[tag]

        # An alternative syntax for setting the color (e.g. <fg red>, <bg red>).
        else:
            if tag.startswith('fg ') or tag.startswith('bg '):
                st, color = tag[:2], tag[3:]
            else:
                st, color = '', tag

            code = '48' if st == 'bg' else '38'

            if st == 'fg' and color in Fore:
                return Fore[color]

            elif st == 'bg' and color in Back:
                return Back[color]
            
            # 8-bits
            elif color.isdigit() and int(color) <= 255:
                return '\033[%s;5;%sm' % (code, color)
            
            # HEX 24-bits
            elif re.match(r'#(?:[a-fA-F0-9]{3}){1,2}$', color):
                hex_color = color[1:]
                if len(hex_color) == 3: hex_color *= 2
                rgb = tuple(int(hex_color[i: i + 2], 16) for i in (0, 2, 4))
                return '\033[%s;2;%s;%s;%sm' % ((code,) + rgb)
            
            # 24-bits
            elif color.count(',') == 2:
                colors = tuple(color.split(', '))
                if all(x.isdigit() and int(x) <= 255 for x in colors):
                    return '\033[%s;2;%s;%s;%sm' % ((code,) + colors)
        return ''

    def colorize(self, text):
        position = 0
        tokens = []
        for i, match in enumerate(self._regex_tag.finditer(text)):
            markup, tag = match.group(0), match.group(1)
            tokens.append(text[position: match.start()])
            if tag[0] == '/':
                token = (
                    Fore.reset if tag.startswith('/fg') else
                    Back.reset if tag.startswith('/bg') else
                    Style.reset_all
                )
            else:
                token = self.get_ansi(tag)
            tokens.append(token)
            position = match.end()

        tokens.append(text[position:])
        return ''.join(tokens)
    
    def strip(self, text):
        return self._regex_tag.sub('', text)
    
    def __repr__(self):
        return 'AnsiColorizer'
    
colorizer = AnsiColorizer()

In [ ]:
show_doc(colorizer)

---

### AnsiColorizer

>      AnsiColorizer (text, fore=None, back=None, styles=None, raw=False,
>                     strip=False)

In [ ]:
show_doc(ColorizedStream)

---

### ColorizedStream

>      ColorizedStream (fore=None, back=None, styles=None, autoreset=True,
>                       streams=('stdout',))

A base class or mixin that enables context managers to work as decorators.

In [ ]:
with ColorizedStream(fore=Fore.dark_violet, autoreset=False):
    print('text1')
    with ColorizedStream(back=Back.light_green, styles=(Style.underline, Style.bold)):
        print('text2')
        with ColorizedStream(fore='red'):
            print('text4')
print('text5')

@ColorizedStream(fore=Fore.dark_orange)
def foo():
    print('FOO')

foo()

text1
text2
text4
text5
FOO


In [ ]:
test_strings = [
    'sdnlfkgn123123123l',
    '<red>testabc</red>--<bg green>test_cdf</bg green>',
    'sdfgdfgs<red>testabc</red>--<fg red><bg green>test_cdf</bg></fg>',
    'sdfgdfgs<50>testabc</fg>--<fg 155><bg 78>test_cdf</bg></fg>',
    '<bold>hsrtjsrtj1214<fg 180, 46, 78>testabc</fg>--<bg 152, 167, 52>test_cdf</bg></bold>',
    'hsrtjsrtj1214<180, 46, 78>testabc</fg>--<152, 167, 52>test_cdf</bg>',
    '<blue>asglskdng---<bg black>alllllg</bg>sdlfnnnn</fg>awesb',
    '<bg #59FFAE>asglskdng---<#AAAA00>alllllg</bg>sdlfnnnn</fg>awesb',
]

for text in test_strings:
    print(colorizer(text))

sdnlfkgn123123123l
testabc--test_cdf
sdfgdfgstestabc--test_cdf
sdfgdfgstestabc--test_cdf
hsrtjsrtj1214testabc--test_cdf
hsrtjsrtj1214testabc--test_cdf
asglskdng---alllllgsdlfnnnnawesb
asglskdng---alllllgsdlfnnnnawesb


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()